# Enviroment

In [ ]:
# ========================================
# 系统级依赖安装 (需root权限)
# ========================================
!apt-get install antiword -qq

# ========================================
# 核心Python包安装
# ========================================
!pip install pymupdf nltk chromadb sentence-transformers gradio -q

# ========================================
# NLP工具初始化
# ========================================
import nltk
nltk.download('punkt_tab')

# ========================================
# 可选组件安装
# ========================================
# 如需Elasticsearch支持可取消注释
# !pip install elasticsearch

# ========================================
# 中文处理扩展
# ========================================
!pip install rank_bm25 jieba

Selecting previously unselected package antiword.
(Reading database ... 126102 files and directories currently installed.)
Preparing to unpack .../antiword_0.37-16_amd64.deb ...
Unpacking antiword (0.37-16) ...
Setting up antiword (0.37-16) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for mailcap (3.70+nmu1ubuntu1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 108.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# ========================================
# 标准库导入
# ========================================
import os
import re
import sys
import time
import datetime
import hashlib
from pathlib import Path
from threading import Thread, Lock
from typing import List, Dict, Tuple
from concurrent.futures import ThreadPoolExecutor, as_completed

# ========================================
# 文件/文档处理
# ========================================
import fitz  # PyMuPDF

# ========================================
# NLP/ML相关库
# ========================================
import nltk
from nltk.tokenize import sent_tokenize
import jieba
import numpy as np
import torch
from torch import cuda
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, CrossEncoder

# ========================================
# 向量数据库/存储
# ========================================
import chromadb
from chromadb.utils import embedding_functions

# ========================================
# 接口/可视化
# ========================================
import gradio as gr
from tqdm import tqdm

# ========================================
# 第三方服务/API
# ========================================
from openai import OpenAI
from tenacity import retry, stop_after_attempt, wait_exponential

# ========================================
# 调试/工具
# ========================================
import traceback

# ========================================
# 已注释的备用依赖
# ========================================
# from elasticsearch import Elasticsearch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Config

In [ ]:
# =============================
# 配置参数（统一管理）
# =============================
class Config:
    # 多线程配置
    NUM_WORKERS = 20
    USE_MULTITHREAD = True

    # 路径配置
    UPLOAD_DIR = "/content/drive/MyDrive/513"
    DB_PATH = "/content/drive/MyDrive/513/chroma_db"

    # 文件配置
    SUPPORTED_EXTS = [".pdf", ".PDF"]  # 支持的文件扩展名
    MAX_FILE_SIZE = 100 * 1024 * 1024  # 100MB文件大小限制

    # 处理参数
    CHUNK_SIZE = 300
    OVERLAP = 50
    TOP_K = 100        # 初始检索数量
    TOP_K_FINAL = 5   # 最终返回数量
    BATCH_SIZE = 16

    # 模型配置
    EMBEDDING_MODEL = "BAAI/bge-m3"
    RERANKER_MODEL = "BAAI/bge-reranker-large"
    API_KEY = "c8bc2c37-95f2-4202-b12d-f9392185408f"
    BASE_URL = "https://ark.cn-beijing.volces.com/api/v3"
    MAX_SEQ_LENGTH = 512

    # 数据库配置
    COLLECTION_NAME = "DocumentDB"
    INSERT_BATCH = 500

    MODELS = {
      "DeepSeek_R1": "deepseek-r1-250120",
      "DeepSeek_V3": "deepseek-v3-250324"
    }
    MAX_TOKENS = 3000
    TEMPERATURE=0.7


# File Processor

## PDF Processor

In [ ]:
# %% ##########################
# PDF处理模块
###############################
class PDFProcessor:
    @staticmethod
    def pdf_to_text(file_path: str) -> str:
        """将PDF文件转换为纯文本"""
        try:
            text = []
            with fitz.open(file_path) as doc:
                for page in doc:
                    text.append(page.get_text())
            return "\n".join(text)
        except Exception as e:
            print(f"PDF解析失败：{file_path}\n错误：{str(e)}")
            return ""

    @staticmethod
    def clean_text(text: str) -> str:
        """文本清洗处理"""
        text = re.sub(r'\n+', ' ', text)  # 合并换行符
        text = re.sub(r'\s+', ' ', text)  # 合并多余空格
        return text.strip()

    @staticmethod
    def batch_process_pdfs(folder_path: str) -> Dict[str, str]:
        """批量处理指定文件夹中的所有PDF文件"""
        pdf_texts = {}
        folder_path = Path(folder_path)

        if not folder_path.is_dir():
            raise ValueError(f"无效的文件夹路径: {folder_path}")

        # 支持递归遍历（可选）
        for pdf_file in folder_path.glob("**/*.pdf"):
            try:
                text = PDFProcessor.pdf_to_text(str(pdf_file))
                cleaned = PDFProcessor.clean_text(text)
                pdf_texts[pdf_file.name] = cleaned
                print(f"成功处理: {pdf_file.name}")
            except Exception as e:
                print(f"处理失败 {pdf_file.name}: {str(e)}")
                continue

        return pdf_texts

### Test

In [ ]:
text_from_pdf = PDFProcessor.pdf_to_text("/content/drive/MyDrive/513/Maotai_2024.pdf")

In [ ]:
text_from_pdf_cleaned = PDFProcessor.clean_text(text_from_pdf)

## Text Chunker

In [ ]:
# %% ##########################
# 文本分块模块
###############################
class TextChunker:

    @staticmethod
    def split_into_chunks(text: str, chunk_size: int) -> List[str]:
      """基于句子的分割"""
      sentences = sent_tokenize(text)
      chunks = []
      current_chunk = []
      current_length = 0

      for sent in sentences:
          sent_len = len(sent)

          # 处理超长单个句子
          if sent_len > chunk_size:
              if current_chunk:  # 先保存已有块
                  chunks.append(" ".join(current_chunk))
                  current_chunk = []
              # 强制分割长句
              chunks.extend([sent[i:i+chunk_size] for i in range(0, sent_len, chunk_size)])
              current_length = 0
              continue

          # 正常累加逻辑
          if current_length + sent_len > chunk_size:
              chunks.append(" ".join(current_chunk))
              # 智能重叠（保留完整句子）
              overlap = []
              overlap_len = 0
              while current_chunk and (overlap_len + len(current_chunk[-1])) <= chunk_size * 0.2:
                  overlap.insert(0, current_chunk.pop())
                  overlap_len += len(overlap[0])
              current_chunk = overlap
              current_length = overlap_len

          current_chunk.append(sent)
          current_length += sent_len

      if current_chunk:
          chunks.append(" ".join(current_chunk))

      return chunks


    # 在TextChunker类中可添加自适应分块算法

    @staticmethod
    def dynamic_chunking(text: str, min_size: int = 200, max_size: int = 500) -> List[str]:
        """
        优化的动态分块算法(支持中英文金融文档)
        Args:
            text: 输入文本(需预先清洗)
            min_size: 最小分块长度(建议200-300)
            max_size: 最大分块长度(建议500-800)
        Returns:
            List[str]: 符合长度限制的语义块列表
        """
        if not text:
            return []

        # 优先级排序的断点符号(分数，符号长度)
        BREAKERS = {
            '\n\n': (0.95, 2),   # 段落分隔
            '。': (0.85, 1),     # 句子结束
            '!': (0.8, 1), '?': (0.8, 1),  # 感叹/疑问
            '；': (0.7, 1), ';': (0.7, 1), # 分号
            '，': (0.5, 1), ',': (0.5, 1) # 逗号
        }

        chunks = []
        start = 0
        text_len = len(text)

        while start < text_len:
            # 计算当前窗口边界
            end = min(start + max_size, text_len)
            window = text[start:end]

            best_split = -1
            best_score = 0

            # 逆向扫描寻找最优断点
            for pos in range(len(window)-1, min_size-1, -1):
                # 获取当前位置上下文
                curr_char = window[pos]
                prev_char = window[pos-1] if pos > 0 else ''
                next_char = window[pos+1] if pos < len(window)-1 else ''

                # 评估所有可能的断点
                for br, (score, br_len) in BREAKERS.items():
                    # 匹配多字符断点
                    if br_len == 2 and pos >= 1:
                        if window[pos-1:pos+1] == br:
                            current_score = score
                            split_pos = pos + 1  # 包含断点符号
                            break
                    elif br_len == 1 and curr_char == br:
                        # 处理数字中的逗号(如1,000)
                        if br in [',', '，'] and next_char.isdigit():
                            continue
                        current_score = score
                        split_pos = pos + 1  # 包含断点符号
                        break
                else:
                    continue  # 未找到断点符号

                # 更新最佳分割点
                if current_score > best_score:
                    best_score = current_score
                    best_split = start + split_pos
                    if best_score >= 0.8:  # 高优先级断点立即确定
                        break

            # 确定最终分割位置
            if best_split > start + min_size:
                chunks.append(text[start:best_split].strip())
                start = best_split
            else:
                # 处理剩余文本不足min_size的情况
                if chunks and (len(window) < min_size):
                    chunks[-1] += window  # 合并到前一个块
                else:
                    chunks.append(window.strip())
                start = end

            # 动态重叠处理(不超过max_size的20%)
            overlap = min(int(max_size * 0.2), start - chunks[-1].rfind('\n\n') if '\n\n' in chunks[-1] else 50)
            start = max(start - overlap, 0)

        return [chunk for chunk in chunks if chunk]


### Test

In [ ]:
text_chunks = TextChunker.split_into_chunks(text_from_pdf_cleaned, Config.CHUNK_SIZE)

In [ ]:
print(len(text_chunks))

565


In [ ]:
print(text_chunks[0])

贵州茅台酒股份有限公司2024 年年度报告 1 / 143 公司代码：600519 公司简称：贵州茅台 贵州茅台酒股份有限公司 2024 年年度报告 贵州茅台酒股份有限公司2024 年年度报告 2 / 143 重要提示 一、 本公司董事会、监事会及董事、监事、高级管理人员保证年度报告内容的真实性、准


In [ ]:
dynamic_text_chunks = TextChunker.dynamic_chunking(text_from_pdf_cleaned)

In [ ]:
print(len(dynamic_text_chunks))

In [ ]:
print(dynamic_text_chunks[0])

# Vector DB

修改权限 防止无法对原来的数据库进行插入

In [ ]:
!find /content/drive/MyDrive/513/chroma_db/ -name "*.lock" -delete

In [ ]:
!chmod -R 775 /content/drive/MyDrive/513/chroma_db/

In [ ]:
# %load_ext line_profiler

# %% ##########################
# 向量数据库模块
###############################
class VectorDBManager:
    #初始化模块
    def __init__(self):
        self.pdf_texter = PDFProcessor()
        self.chunker = TextChunker()
        self.client = self._init_client()
        self.embedder = self._init_embedder()
        self.collection = self._init_collection()
        self.insert_executor = ThreadPoolExecutor(max_workers=Config.NUM_WORKERS)

    def _init_embedder(self):
        """初始化嵌入模型（可并行优化点）"""
        model = SentenceTransformer(Config.EMBEDDING_MODEL)
        model.max_seq_length = Config.MAX_SEQ_LENGTH
        return model

    def _init_client(self):
        settings = chromadb.config.Settings(
          persist_directory=Config.DB_PATH,
          anonymized_telemetry=False,
          allow_reset=True,  # 允许重置数据库
          # enable_lock=False   # 禁用文件锁（针对Colab环境）
          )

        return chromadb.PersistentClient(settings=settings)


    def _init_collection(self):
        """初始化数据库集合"""


        embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
            model_name=Config.EMBEDDING_MODEL,
            device="cuda",
            normalize_embeddings=True
        )
        return self.client.get_or_create_collection(
            name=Config.COLLECTION_NAME,
            embedding_function=embedding_func,
            metadata={"hnsw:space": "cosine"},
        )

    #生成向量
    def pdf_processor(self, pdf_path: str):
        # 步骤1：PDF转文本
        print("正在解析PDF...")
        raw_text = self.pdf_texter.pdf_to_text(pdf_path)
        if not raw_text:
          print("PDF解析失败，请检查文件格式")
          return

        # 步骤2：文本清洗和分块
        cleaned_text = self.pdf_texter.clean_text(raw_text)
        chunks = self.chunker.split_into_chunks(cleaned_text, Config.CHUNK_SIZE)
        chunks = [c for c in chunks if len(c.strip()) > 0]  # 过滤空块
        print(f"生成有效文本块：{len(chunks)} 个")

        # 步骤3：生成向量（可并行优化点）
        print("生成嵌入向量...")
        embeddings = []
        metadata = []
        doc_id = Path(pdf_path).stem  # 获取文件名（不含扩展名）

        if Config.USE_MULTITHREAD:
          #多线程版本
          print("多线程版本\n")
          # 修改后的向量生成部分
          def process_batch(batch, start_idx, doc_id):
              """单批次处理函数（线程安全）"""
              # 生成向量
              vectors = self.embedder.encode(
                  batch,
                  convert_to_tensor=True,
                  normalize_embeddings=True,
                  show_progress_bar=False
              ).cpu().numpy()

              # 构建元数据
              batch_meta = [{
                  "doc_id": doc_id,
                  "chunk_index": start_idx + idx,
                  "text_snippet": chunk[:100] + "...",
              } for idx, chunk in enumerate(batch)]

              return vectors, batch_meta

          with ThreadPoolExecutor(max_workers=Config.NUM_WORKERS) as executor:
            # 1. 准备任务队列
            futures = []
            for i in range(0, len(chunks), Config.BATCH_SIZE):
                batch = chunks[i:i+Config.BATCH_SIZE]
                # 提交任务（立即非阻塞）
                future = executor.submit(
                    process_batch,
                    batch=batch,
                    start_idx=i,
                    doc_id=doc_id
                )
                futures.append(future)

            # 2. 并行处理 + 进度跟踪
            for future in tqdm(as_completed(futures),
                              total=len(futures),
                              desc="多线程生成向量"):
                batch_vectors, batch_meta = future.result()
                embeddings.extend(batch_vectors)
                metadata.extend(batch_meta)
        else:
          #单线程版本
          print("单线程版本\n")
          for i in tqdm(range(0, len(chunks), Config.BATCH_SIZE), desc="向量生成"):
              batch = chunks[i:i+Config.BATCH_SIZE]

              # 生成向量
              vectors = self.embedder.encode(
                  batch,
                  convert_to_tensor=True,
                  normalize_embeddings=True,
                  show_progress_bar=False
              ).cpu().numpy()

              # 收集数据
              embeddings.extend(vectors)
              metadata.extend([{
                  "doc_id": doc_id,
                  "chunk_index": i + idx,
                  "text_snippet": chunk[:100] + "...",  # 截取前100字符
              } for idx, chunk in enumerate(batch)])

        # 步骤4：存储到数据库
        print("\n正在写入数据库...")

        if Config.USE_MULTITHREAD:
          self.insert_data_parallel(chunks, embeddings, metadata)
        else:
          self.insert_data(chunks, embeddings, metadata)

    #向量插入
    @retry(stop=stop_after_attempt(3),
           wait=wait_exponential(multiplier=1, min=2, max=10))
    def _insert_batch(self, batch: dict):
        try:
          self.collection.add(**batch)
          return True
        except Exception as e:
          print(f"插入重试中... 错误：{str(e)}")
          raise

    def insert_data(self, chunks: List[str], embeddings: List[np.ndarray], metadata: List[Dict]):
        """批量插入数据（包含完整文本内容）

        参数说明：
        - chunks: 文本块列表（原始文本内容）
        - embeddings: 对应的向量列表
        - metadata: 元数据列表（需包含doc_id等信息）
        """
        # 数据校验（确保三个列表长度一致）
        assert len(chunks) == len(embeddings) == len(metadata), "数据长度不一致"

        # 准备数据格式（关键修改点）
        ids = [f"{m['doc_id']}_{m['chunk_index']}" for m in metadata]  # 唯一ID
        documents = chunks  # 直接使用原始文本块作为文档内容

        # 分批次插入（显示进度条）
        for i in tqdm(range(0, len(ids), Config.INSERT_BATCH),
                    desc="数据库插入",
                    unit="batch"):
            batch = {
                "ids": ids[i:i+Config.INSERT_BATCH],
                "embeddings": embeddings[i:i+Config.INSERT_BATCH],
                "documents": documents[i:i+Config.INSERT_BATCH],  # 关键修改
                "metadatas": metadata[i:i+Config.INSERT_BATCH]
            }
            try:
                self.collection.add(**batch)
            except Exception as e:
                print(f"插入失败批次 {i}-{i+Config.INSERT_BATCH}: {str(e)}")

        print(f"插入完成，总数据量：{self.collection.count()}")

    def insert_data_parallel(self, chunks: List[str], embeddings: List[np.ndarray], metadata: List[Dict]):
        """并行化批量插入"""
        assert len(chunks) == len(embeddings) == len(metadata)

        # 准备所有批次数据（避免在并行中修改）
        batch_list = []
        ids = [f"{m['doc_id']}_{m['chunk_index']}" for m in metadata]
        for i in range(0, len(ids), Config.INSERT_BATCH):
            batch_list.append({
                "ids": ids[i:i+Config.INSERT_BATCH],
                "embeddings": embeddings[i:i+Config.INSERT_BATCH],
                "documents": chunks[i:i+Config.INSERT_BATCH],
                "metadatas": metadata[i:i+Config.INSERT_BATCH]
            })

        # 并行提交任务
        futures = []
        for batch in batch_list:
            futures.append(self.insert_executor.submit(self._insert_batch, batch))

        # 监控进度与错误
        success_count = 0
        with tqdm(as_completed(futures), total=len(batch_list), desc="并行插入") as pbar:
            for future in pbar:
                try:
                    if future.result():
                        success_count += 1
                    pbar.set_postfix_str(f"成功: {success_count}/{len(batch_list)}")
                except Exception as e:
                    print(f"\n永久插入失败: {str(e)}")

        print(f"插入完成 - 成功批次: {success_count}/{len(batch_list)}")

    def search(self, query: str, top_k: int = Config.TOP_K) -> List[Tuple[str, float, dict]]:
        """增强型向量搜索（返回完整元数据）"""
        results = self.collection.query(
            query_texts=[query],
            n_results=top_k,
            include=["documents", "distances", "metadatas"]
        )
        return list(zip(
            results["documents"][0],
            results["distances"][0],
            results["metadatas"][0]
        ))

    # 资源检查
    # def _check_resources(self):
    #     """系统资源检查"""
    #     # 存储空间检查
    #     free_space = psutil.disk_usage(Config.DB_PATH).free
    #     if free_space < 1 * 1024**3:  # 1GB
    #         raise RuntimeError("存储空间不足，需要至少1GB可用空间")

    #     # 内存检查
    #     mem = psutil.virtual_memory()
    #     if mem.available < 2 * 1024**3:  # 2GB
    #         raise RuntimeError("内存不足，需要至少2GB可用内存")

    def process_folder(self, folder_path: str):
        """批量处理文件夹中的PDF文件"""
        print(f"开始批量处理文件夹: {folder_path}")
        start_time = time.time()

        # 获取所有PDF文件
        pdf_files = [f for f in Path(folder_path).glob("*.pdf") if f.is_file()]
        if not pdf_files:
            raise FileNotFoundError("未找到PDF文件")

        # 使用线程池并行处理
        with ThreadPoolExecutor(max_workers=Config.NUM_WORKERS) as executor:
            futures = []
            for pdf_file in pdf_files:
                future = executor.submit(self.pdf_processor, str(pdf_file))
                futures.append(future)

            # 处理完成统计
            success = 0
            for future in tqdm(as_completed(futures), total=len(futures), desc="批量处理"):
                try:
                    future.result()
                    success += 1
                except Exception as e:
                    print(f"处理失败: {str(e)}")

        print(f"处理完成 成功: {success}/{len(pdf_files)}")
        print(f"总耗时: {time.time()-start_time:.2f}秒")


## Test

In [ ]:
text_db = VectorDBManager()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
text_db.pdf_processor("/content/drive/MyDrive/513/Maotai_2024.pdf")

正在解析PDF...
生成有效文本块：565 个
生成嵌入向量...
多线程版本



多线程生成向量: 100%|██████████| 36/36 [00:18<00:00,  1.94it/s]



正在写入数据库...


并行插入: 100%|██████████| 2/2 [00:08<00:00,  4.20s/it, 成功: 2/2]

插入完成 - 成功批次: 2/2


In [ ]:
def process_pdfs(folder_path: str):

    files = [f for f in os.listdir(folder_path)]

    for file in files:
            try:
                # 构建完整路径
                file_path = os.path.join(folder_path, file)
                text_db.pdf_processor(file_path)
            except Exception as e:
                print(f"处理失败：{file} | 错误：{str(e)}")

In [ ]:
process_pdfs("/content/drive/MyDrive/513/A50_pdfs_named/")

正在解析PDF...
生成有效文本块：427 个
生成嵌入向量...
多线程版本



多线程生成向量: 100%|██████████| 27/27 [00:14<00:00,  1.89it/s]



正在写入数据库...


并行插入: 100%|██████████| 1/1 [00:01<00:00,  1.36s/it, 成功: 1/1]


插入完成 - 成功批次: 1/1
正在解析PDF...
生成有效文本块：405 个
生成嵌入向量...
多线程版本



...

插入完成 - 成功批次: 3/3
正在解析PDF...
生成有效文本块：584 个
生成嵌入向量...
多线程版本



多线程生成向量: 100%|██████████| 37/37 [00:20<00:00,  1.83it/s]



正在写入数据库...


并行插入: 100%|██████████| 2/2 [00:07<00:00,  3.61s/it, 成功: 2/2]

插入完成 - 成功批次: 2/2


In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
answers = text_db.search("贵州茅台2024年净利润")

In [ ]:
print(len(answers))

100


In [ ]:
print(answers[0])

('贵州茅台酒股份有限公司2024 年年度报告 1 / 143 公司代码：600519 公司简称：贵州茅台 贵州茅台酒股份有限公司 2024 年年度报告 贵州茅台酒股份有限公司2024 年年度报告 2 / 143 重要提示 一、 本公司董事会、监事会及董事、监事、高级管理人员保证年度报告内容的真实性、准确性、 完整性，不存在虚假记载、误导性陈述或重大遗漏，并承担个别和连带的法律责任。 二、 公司全体董事出席董事会会议。 三、 天健会计师事务所(特殊普通合伙)为本公司出具了标准无保留意见的审计报告。 四、 公司负责人张德芹、主管会计工作负责人蒋焰及会计机构负责人（会计主管人员）蔡聪应声 明：保证年度报告中财务报告的真实、准确、完整。 五、 董事会决议通过的本报告期利润分配预案或公积金转增股本预案 公司拟以实施权益分派股权登记日登记的总股本扣除回购专用账户中的股份为基数，实施 2024年年度利润分配，向全体股东每10股派发现金红利276.24元（含税）。截至2025年3月31日， 公司总股本1,256,197,800股，回购专用证券账户中股份数1,082,700股，总股本扣除回购专用证 券账户中的股份数为1,255,115,100股，以此计算合计派发现金红利34,671,299,522.40元（含税 ）。在实施权益分派的股权登记日前公司总股本扣除回购专用证券账户内的股份数如发生变动的 ，将维持分配总额不变，相应调整每股分红比例。以上利润分配预案需提交公司股东大会审议通 过后实施。 六、 前瞻性陈述的风险声明 √适用 □不适用 本年度报告中所涉及的未来计划、发展战略等前瞻性陈述因存在不确定性，不构成本公司对 投资者的实质承诺，敬请投资者注意投资风险。 七、 是否存在被控股股东及其他关联方非经营性占用资金情况 否 八、 是否存在违反规定决策程序对外提供担保的情况 否 九、 是否存在半数以上董事无法保证公司所披露年度报告的真实性、准确性和完整性 否 十、 重大风险提示 本公司已在本年度报告中“公司关于公司未来发展的讨论与分析”章节阐述公司可能面对的 风险，敬请投资者予以关注。 十一、 其他 □适用 √不适用 贵州茅台酒股份有限公司2024 年年度报告 3 / 143 目录 第一节 释义 .............................................

# Retriever

In [ ]:
# =============================
# 检索结果处理模块
# =============================
class RetrievalProcessor:
    """处理检索结果的重排序和过滤"""
    def __init__(self):
        try:
            self.reranker = CrossEncoder(Config.RERANKER_MODEL)
            self.bm25 = None  # 延迟初始化
        except Exception as e:
            print(f"RetrievalProcessor初始化错误:{str(e)}")
            raise

    def process_results(self, query: str, results: List[Tuple[str, float, dict]]):
        """
        处理检索结果的三阶段流程：
        1. BM25过滤 -> 2.重排序 -> 3.元数据过滤
        """
        # 阶段1：BM25过滤
        try:
            filtered_indices = self._bm25_filter(query, [r[0] for r in results])
            filtered_results = [results[i] for i in filtered_indices]  # 新增结果过滤
        except Exception as e:
            return f"处理检索结果的阶段1：BM25过滤错误:{str(e)}"

        # 阶段2：重排序
        try:
            # 修正点：传入原始结果和过滤后的索引
            reranked = self._rerank(query, filtered_results, filtered_indices)
        except Exception as e:
            return f"阶段2：重排序错误:{str(e)}"

        # 阶段3：元数据过滤
        try:
            final_results = self._metadata_filter(reranked)
        except Exception as e:
            return f"阶段3：元数据过滤错误:{str(e)}"

        return final_results[:Config.TOP_K_FINAL]

    def _bm25_filter(self, query: str, documents: List[str]) -> List[int]:
        """使用BM25进行关键词过滤"""
        # 初始化BM25
        try:
            # 添加空文档检查
            valid_docs = [doc for doc in documents if len(doc.strip()) > 0]
            if not valid_docs:
                return []

            tokenized_docs = [jieba.lcut_for_search(doc) for doc in valid_docs]
            self.bm25 = BM25Okapi(tokenized_docs)
        except Exception as e:
            raise Exception(f"初始化BM25错误:{str(e)}")

        # 提取查询关键词
        try:
            query_terms = jieba.lcut_for_search(query.strip())
            if not query_terms:
                return []
        except Exception as e:
            raise Exception(f"提取查询关键词错误:{str(e)}")

        # 获取BM25分数
        try:
            scores = self.bm25.get_scores(query_terms)
        except Exception as e:
            raise Exception(f"获取BM25分数错误:{str(e)}")

        # 选取前50%的文档（修正索引映射）
        threshold = np.percentile(scores, 50)
        return [i for i, s in enumerate(scores) if s >= threshold]

    def _rerank(self, query: str, filtered_results: List[Tuple], indices: List[int]) -> List[Tuple]:
        """使用重排序模型优化结果"""
        # 修正点：使用过滤后的结果数据
        pairs = [(query, res[0]) for res in filtered_results]  # ✅ 使用传入的filtered_results
        try:
            scores = self.reranker.predict(pairs, batch_size=32)
        except Exception as e:
            raise Exception(f"重排序模型预测错误:{str(e)}")

        # 组合结果时保留原始元数据
        reranked = sorted(
            zip(filtered_results, scores),
            key=lambda x: -x[1]
        )
        return reranked

    def _metadata_filter(self, results: List[Tuple]) -> List[Tuple]:
        """基于元数据的过滤（修正文本长度检查）"""
        # 修正点：正确访问文本内容
        return [r for r in results if len(r[0][0].strip()) >= 50]  # ✅ r[0][0]对应原始文本内容

## Test

In [ ]:
test_retriever = RetrievalProcessor()

config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/34.0k [00:00<?, ?B/s]

In [ ]:
final_answers = test_retriever.process_results("贵州茅台2024年净利润", answers)

In [ ]:
print(len(final_answers))

1


In [ ]:
print(final_answers[:])

[(('事项描述 相关信息披露详见审计报告财务报表附注三(二十)和五(二)1。 贵州茅台酒股份有限公司2024 年年度报告 56 / 143 贵州茅台公司的营业收入主要来自于茅台酒及系列酒的生产及销售。2024 年度，贵州茅台 公司的营业收入为人民币17,089,915.23 万元。其中主营业务收入为人民币17,061,183.81 万 元，占营业收入的99.83%。 由于营业收入是贵州茅台公司关键业绩指标之一，可能存在贵州茅台公司管理层（以下简称 管理层）通过不恰当的收入确认以达到特定目标或预期的固有风险，同时，收入确认涉及重大管 理层判断。因此，我们将收入确认确定为关键审计事项。 2. 审计应对 针对收入确认，我们实施的审计程序主要包括： (1) 了解与收入确认相关的关键内部控制，评价这些控制的设计，确定其是否得到执行，并 测试相关内部控制的运行有效性； (2) 选取样本检查销售合同，识别与客户取得相关商品控制权时点相关的条款，评价收入确 认政策是否符合企业会计准则的规定； (3) 对营业收入及毛利率实施分析程序，识别是否存在重大或异常波动，并查明波动原因； (4) 对于内销收入，选取样本检查与收入确认相关的支持性文件，包括销售合同、销售订 单、出库单、签收单、销售发票等；对于出口外销收入，选取样本检查与收入确认相关的支持性 文件，包括销售合同、销售订单、出口报关单、货运提单等； (5) 结合合同负债函证，选取项目向主要客户函证报告期销售额； (6) 实施截止测试，检查收入是否在恰当期间确认； (7) 检查与营业收入相关的信息是否已在财务报表中作出恰当列报。 (二) 关联方及其交易 1. 事项描述 相关信息披露详见审计报告财务报表附注十一。 2024 年度贵州茅台公司与关联方发生交易的类别较多且金额较大。由于关联方数量多、交 易类型多样、交易金额重大，关联交易金额准确性对本年度财务报表影响较大，且披露的完整性 对理解和使用财务报表具有较大影响，我们将其确认确定为关键审计事项。 2. 审计应对 (1) 了解贵州茅台公司与识别和披露关联方及关联交易相关的关键内部控制，评价这些控制 的设计，确定其是否得到执行，并测试相关内部控制运行的有效性； (2) 获取管理层提供的关联方清单，与通过公开渠道查询的信息进行核对； (3) 获取关联交易相关的董事会决议、股东大会决议，检

In [ ]:
print(final_answers[0][0])

('事项描述 相关信息披露详见审计报告财务报表附注三(二十)和五(二)1。 贵州茅台酒股份有限公司2024 年年度报告 56 / 143 贵州茅台公司的营业收入主要来自于茅台酒及系列酒的生产及销售。2024 年度，贵州茅台 公司的营业收入为人民币17,089,915.23 万元。其中主营业务收入为人民币17,061,183.81 万 元，占营业收入的99.83%。 由于营业收入是贵州茅台公司关键业绩指标之一，可能存在贵州茅台公司管理层（以下简称 管理层）通过不恰当的收入确认以达到特定目标或预期的固有风险，同时，收入确认涉及重大管 理层判断。因此，我们将收入确认确定为关键审计事项。 2. 审计应对 针对收入确认，我们实施的审计程序主要包括： (1) 了解与收入确认相关的关键内部控制，评价这些控制的设计，确定其是否得到执行，并 测试相关内部控制的运行有效性； (2) 选取样本检查销售合同，识别与客户取得相关商品控制权时点相关的条款，评价收入确 认政策是否符合企业会计准则的规定； (3) 对营业收入及毛利率实施分析程序，识别是否存在重大或异常波动，并查明波动原因； (4) 对于内销收入，选取样本检查与收入确认相关的支持性文件，包括销售合同、销售订 单、出库单、签收单、销售发票等；对于出口外销收入，选取样本检查与收入确认相关的支持性 文件，包括销售合同、销售订单、出口报关单、货运提单等； (5) 结合合同负债函证，选取项目向主要客户函证报告期销售额； (6) 实施截止测试，检查收入是否在恰当期间确认； (7) 检查与营业收入相关的信息是否已在财务报表中作出恰当列报。 (二) 关联方及其交易 1. 事项描述 相关信息披露详见审计报告财务报表附注十一。 2024 年度贵州茅台公司与关联方发生交易的类别较多且金额较大。由于关联方数量多、交 易类型多样、交易金额重大，关联交易金额准确性对本年度财务报表影响较大，且披露的完整性 对理解和使用财务报表具有较大影响，我们将其确认确定为关键审计事项。 2. 审计应对 (1) 了解贵州茅台公司与识别和披露关联方及关联交易相关的关键内部控制，评价这些控制 的设计，确定其是否得到执行，并测试相关内部控制运行的有效性； (2) 获取管理层提供的关联方清单，与通过公开渠道查询的信息进行核对； (3) 获取关联交易相关的董事会决议、股东大会决议，检查关

# Chat System

In [ ]:
class ChatSystem:
    def __init__(self):
        self.client = OpenAI(
            base_url=Config.BASE_URL,
            api_key=Config.API_KEY
        )

    def generate_response(self, context: str, query: str, model_name: str) -> str:
        """生成智能回复"""
        prompt = f"""
        基于以下上下文信息，请用中文回答问题。如果无法从上下文中得到答案，请说明原因。

        上下文：
        {context}

        问题：{query}
        """
        try:
          response = self.client.chat.completions.create(
              model=model_name,
              messages=[
                  {"role": "system", "content": "你是一个专业的财务报表分析人工智能助手,请适当采用语法高亮对输出进行美化"},
                  {"role": "user", "content": prompt}
              ],
              temperature=Config.TEMPERATURE,
              max_tokens=Config.MAX_TOKENS
          )
        except Exception as e:
          return f"{str(e)}"

        return response.choices[0].message.content

## Test

In [ ]:
chat = ChatSystem()

In [ ]:
chat.generate_response(final_answers[0][0], "贵州茅台2024年净利润", "deepseek-r1-250120")

'\n\n根据贵州茅台2024年年度报告中的合并利润表数据，**贵州茅台2024年净利润为**：\n\n```python\n净利润 = 89,334,728,025.90 元（约893.35亿元人民币）\n```\n\n**具体分类：**\n- **归属于母公司股东的净利润**：86,228,146,421.62 元（约862.28亿元）\n- **少数股东损益**：3,106,581,604.28 元（约31.07亿元）\n\n---\n\n**数据来源：**\n- 合并利润表中“五、净利润”项目（报表页码：64-66页）\n- 净利润构成中明确列示了母公司股东和少数股东的分配比例。'

# User Interface

In [ ]:
# =============================
# 交互界面模块
# =============================
class ChatInterface:
    """Gradio交互界面管理"""
    def __init__(self):
      try:
         self.db = VectorDBManager()
         self.retriever = RetrievalProcessor()
         self.chat_system = ChatSystem()
         self._setup_ui()
      except Exception as e:
        print(f"界面初始化失败:{str(e)}")
        raise

    # 在ChatInterface类中修改handle_upload方法
    def handle_upload(self, file_obj) -> str:
        """支持处理文件夹上传"""
        try:
            if isinstance(file_obj, list):  # Gradio文件夹上传会返回列表
                for f in file_obj:
                    self.db.pdf_processor(f.name)
                return f"成功处理 {len(file_obj)} 个文件"
            else:
                self.db.pdf_processor(file_obj.name)
                return f"成功处理: {Path(file_obj.name).name}"
        except Exception as e:
            return f"处理失败: {str(e)}"

    def _setup_ui(self):
        """构建交互界面"""
        with gr.Blocks(title="智能文档助手🤖") as self.interface:
          with gr.Row():
            with gr.Column(scale=1):
              # # 多文件/文件夹上传
              # self.file_upload = gr.File(
              #     label="上传PDF文档/文件夹",
              #     file_count="directory"  # 支持文件夹上传
              # )
              self.file_upload = gr.File(label="上传PDF文档")
              self.upload_btn = gr.Button("处理文档")
              self.status = gr.Textbox(label="处理状态")

            with gr.Column(scale=2):
              # 模型选择组件
              self.model_selector = gr.Dropdown(
                  label="选择模型",
                  info="Deepseek-R1:深度推理 | Deepseek-V3:快速响应",
                  choices=list(Config.MODELS.keys()),
                  value="Deepseek-V3"
              )


              # 问答组件
              self.query_input = gr.Textbox(label="输入问题")
              self.search_btn = gr.Button("开始查询")
              self.context_display = gr.Textbox(label="检索上下文")
              self.answer_output = gr.Textbox(label="智能回复")

            # 事件绑定
            self.upload_btn.click(
                self.handle_upload,
                inputs=self.file_upload,
                outputs=self.status
            )
            self.search_btn.click(
                self.handle_query,
                inputs=[self.query_input, self.model_selector],
                outputs=[self.context_display, self.answer_output]
            )

    def handle_upload(self, file_path: str) -> str:
        """处理文件上传"""
        if not file_path:
            return "请选择PDF文件"

        try:
          self.db.pdf_processor(file_path)
          return f"✅ 成功处理: {Path(file_path).name}"
        except Exception as e:
          return f"❌ 文件处理失败，请检查文件格式. {str(e)}"

    def handle_query(self, query: str, model: str) -> Tuple[str, str]:
        """处理用户查询"""
        model_id = Config.MODELS.get(model)

        # 向量搜索
        raw_results = self.db.search(query)

        # 结果处理
        processed = self.retriever.process_results(query, raw_results)

        context = "\n".join([r[0][0] for r in processed])

        # 生成智能回复
        response = self.chat_system.generate_response(context, query, model_id)

        return context, response

## Test

In [ ]:
UI = ChatInterface()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/34.0k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/gradio/components/dropdown.py:227: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: Deepseek-V3 or set allow_custom_value=True.
  warnings.warn(


In [ ]:
UI.interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://64d8effc465377dccb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
